In [13]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import requests
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [3]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("arnav.thakrar@gmail.com")  # Change to your verified sender
    to_email = To("arnav.thakrar@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [4]:
@function_tool
def send_html_email(subject: str, html_body: str, recipient: str) -> Dict[str, str]:
    """
    Send an HTML email using SendGrid.

    Args:
        subject (str): The subject line of the email.
        html_body (str): The HTML content of the email.
        recipient (str): Recipient email address.

    Returns:
        Dict[str, str]: Status of the email sending operation.
    """
    try:
        api_key = os.environ.get("SENDGRID_API_KEY")
        if not api_key:
            return {"status": "error", "message": "SendGrid API key not set in environment"}

        sg_client = sendgrid.SendGridAPIClient(api_key=api_key)
        from_email = Email("arnav.thakrar@gmail.com")  # Must be verified in SendGrid
        to_email = To(recipient)
        content = Content("text/html", html_body)

        mail = Mail(from_email, to_email, subject, content)
        response = sg_client.client.mail.send.post(request_body=mail.get())

        if 200 <= response.status_code < 300:
            return {"status": "success", "message": f"Email sent to {recipient}"}
        else:
            return {
                "status": "error",
                "message": f"SendGrid responded with status {response.status_code}",
                "body": response.body.decode() if hasattr(response.body, 'decode') else str(response.body)
            }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [5]:
instructions = """
You are a News Email Agent. Your task is to take a Markdown-formatted news summary provided by the user, 
convert it to a clean HTML email, and send it to the recipient email address specified in the user's prompt.

Follow these rules carefully:

1. **Input Handling**:
   - The user will provide a news summary in Markdown format.
   - The user will also provide a recipient email address where the email should be sent.

2. **Markdown to HTML Conversion**:
   - Convert all Markdown elements to proper HTML:
       - Headings (#, ##, ###) → <h1>, <h2>, <h3>
       - Bold (**text**) → <strong>
       - Italics (*text*) → <em>
       - Links ([text](url)) → <a href="url">text</a>
       - Lists (- item or * item) → <ul><li>item</li></ul>
       - Paragraphs → <p>
   - Ensure the resulting HTML is clean and readable in an email client.

3. **Email Composition**:
   - Set the email subject line as "Daily News Summary".
   - Use the HTML content generated from the Markdown conversion as the body.

4. **Call the Tool**:
   - Use the `send_html_email` function tool.
   - Pass the following arguments:
       - `subject`: "Daily News Summary"
       - `html_body`: the HTML you generated
       - `recipient`: the email address provided by the user

5. **Error Handling**:
   - If the tool reports an error, provide a clear message to the user with the reason.
   - Confirm successful email sending to the user with the recipient address.

6. **Output**:
   - Never send raw Markdown; always convert it to HTML before sending.
   - Provide a short confirmation message to the user about email delivery status.
"""


In [6]:
news_markdown = """
# News Digest

## Google & AI Updates

### Pixel 10 Series Teasers Highlight AI Features
Google has begun teasing new AI-powered features for its upcoming Pixel 10 smartphones, emphasizing advances in telephoto camera capabilities and an improved "Add Me" group photo feature—an easy-to-use AI tool first seen with the Pixel 9, which allows the photographer to appear in group photos via image merging.

The teaser video also hinted at Google's advancements in AI integration, particularly through its Gemini AI, with more details expected at the Made by Google event on August 20 in New York City. The event will feature notable guests such as Jimmy Fallon, Stephen Curry, and Lando Norris.

Rumors suggest the Pixel 10 Pro Fold may include upgrades like Qi2 wireless charging and possibly a dustproof IP68 rating, addressing earlier foldable device limitations.

### Pixel 9 Pro Fold Discounts
As Google prepares for the Pixel 10 launch, the Pixel 9 Pro Fold is now offered at a significant discount (down 33% to $1,199), presenting a good value despite minor camera limitations compared to its non-foldable counterpart.

The Pixel 10 Pro Fold is expected to build on its predecessor's design and potentially resolve durability concerns, making it one to watch in the foldable phone market.

## Other Technology & Deal Highlights
- **Lenovo Legion Go S**: Now $499.99 (down $300), the Windows handheld gaming device gets an attractive price cut, though performance and Windows integration remain questionable.  
- **Sonos One SL Refurbished**: Available for $119 with promo code, this speaker offers strong value for those seeking to expand audio setups without voice assistant features.  
- **4K Blu-ray Movie Sale**: Select films available in a “3 for $30” offer through Gruv, appealing to physical media collectors.

## Trump & The Economy
(No articles provided on Trump or the economy in this news cycle.)

*Stay tuned for more updates on Google's AI advancements and future device launches as details emerge from their August 20 event.*
"""

recipient_email = "arnav.thakrar@gmail.com"

In [10]:
prompt = f"""
You are receiving a personalized news summary in Markdown format and need to send it as an HTML email.

Here is the news summary (Markdown format):
{news_markdown}

Please send this news summary as an HTML email to the recipient email address below:

Recipient Email: {recipient_email}

Instructions for yourself:
1. Convert the Markdown content above into proper HTML suitable for an email.
2. Set the email subject to "Daily News Summary".
3. Call the `send_html_email` function tool with the following parameters:
   - subject: "Daily News Summary"
   - html_body: the HTML content you generated
   - recipient: {recipient_email}
4. Report back whether the email was successfully sent or if there was an error.
"""


In [11]:
email_agent = Agent(
    name="Email Agent",
    instructions=instructions,
    tools=[send_html_email],
    model="gpt-4.1",
)

In [14]:
with trace("News Summary"):
    result = await Runner.run(email_agent, input=prompt)
    display(Markdown(result.final_output))

The Daily News Summary has been successfully sent as an HTML email to arnav.thakrar@gmail.com. If you have any more summaries or need further assistance, feel free to ask!